In [1]:
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import mean_squared_error,mean_squared_log_error,r2_score
from sklearn_pandas import DataFrameMapper
from pandas.api.types import is_string_dtype, is_numeric_dtype

from scipy.stats import skew
from scipy.special import boxcox1p

%matplotlib inline

In [2]:
def get_cat_columns_by_type(df):
    out = []
    for colname,col_values in df.items():
        if is_string_dtype(col_values):
            out.append((colname,'string') )
        elif not is_numeric_dtype(col_values):
            out.append((colname,'categorical') )
    return out       

def get_numeric_columns(df):
    out = []
    for colname,col_values in df.items():
        if is_numeric_dtype(col_values):
            out.append(colname)
    return out       
    
def get_missing_values_percentage(df):
    missing_values_counts_list = df.isnull().sum()
    total_values = np.product(df.shape)
    total_missing = missing_values_counts_list.sum()
    # percent of data that is missing
    return (total_missing/total_values) * 100


def convert_to_str_type(df_in,columns,inplace=False):
    if(inplace):
        df = df_in
    else:
        df = df_in.copy()
        
    for col in columns:
        df[col] = df[col].astype(str)
    return df

    
def handle_missing_values(df_in,cat_cols=[], num_cols=[],na_dict=None,add_nan_col=True,inplace=False):
    if(inplace):
        df = df_in
    else:
        df = df_in.copy()
 
    if na_dict is None:
        na_dict = {}

    for colname, col_values in df.items():   
        if colname not in num_cols:
            continue
        if pd.isnull(col_values).sum():
            df[colname+'_na'] = pd.isnull(col_values)
            filler = na_dict[colname] if colname in na_dict else col_values.median()
            df[colname] = col_values.fillna(filler)
            na_dict[colname] = filler
    for colname in cat_cols:
        df[colname].fillna(df[colname].mode()[0], inplace=True)
        lbl = LabelEncoder() 
        lbl.fit(list(df[colname].values)) 
        df[colname] = lbl.transform(list(df[colname].values))
    
    return (df,na_dict)



def scale_num_cols(df_in, mapper, inplace=False):
    if(inplace):
        df = df_in
    else:
        df = df_in.copy()
        
    if mapper is None:
        map_f = [([c],StandardScaler()) for c in df.columns if is_numeric_dtype(df[c])]
        mapper = DataFrameMapper(map_f).fit(df)
    df[mapper.transformed_names_] = mapper.transform(df)
    return (df,mapper)



def extract_and_drop_target_column(df_in, y_name, inplace=False):
    if(inplace):
        df = df_in
    else:
        df = df_in.copy()
    if not is_numeric_dtype(df[y_name]):
        df[y_name] = df[y_name].cat.codes
        y = df[y_name].values
    else:
        y = df[y_name]
    df.drop([y_name], axis=1, inplace=True)
    return (df,y)

def print_mse(m,X_train, X_valid, y_train, y_valid):
    res = [mean_squared_error(y_train,m.predict(X_train)),
                mean_squared_error(y_valid,m.predict(X_valid)),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    print('MSE Training set = {}, MSE Validation set = {}, score Training Set = {}, score on Validation Set = {}'.format(res[0],res[1],res[2], res[3]))
    if hasattr(m, 'oob_score_'):
          print('OOB Score = {}'.format(m.oob_score_))      

def get_iqr_min_max(df,cols):
    out = {}
    for colname, col_values in df.items():
        if colname not in cols:
            continue
        quartile75, quartile25 = np.percentile(col_values, [75 ,25])
        ## Inter Quartile Range ##
        IQR = quartile75 - quartile25
        min_value = quartile25 - (IQR*1.5)
        max_value = quartile75 + (IQR*1.5)
        out[colname] = (min_value,max_value)
    return out


def bin_numerical_columns(df_in,cols,inplace=False):
    if(inplace):
        df = df_in
    else:
        df = df_in.copy()
        
    for col in cols.keys():
        bins = cols[col]
        buckets_ = np.linspace(bins[0],bins[1],bins[2])
        df[col] = pd.cut(df[col],buckets_,include_lowest=True)
    return df

In [3]:
def preprocess_df(df_train,df_test=None,
                  log_y=True,
                  id_col= None,
                  drop_target=True,
                 convert_to_cat_cols=None,
                 remove_skewness=False,scale_mapper=None,
                 bin_columns_dict=None):
    
    if drop_target:
        df,y = extract_and_drop_target_column(df_train,'SalePrice',inplace=True)
    if log_y:
        y = np.log1p(y)
    else:
        y = None
        
    combined = pd.concat((df, df_test)).reset_index(drop=True)
    
    if df_test is not None and id_col is not None:
        test_id = df_test['Id'].copy()
        combined.drop('Id', axis=1,inplace=True)
    else: test_id = None
    
    if convert_to_cat_cols is not None:
        combined = convert_to_str_type(combined,convert_to_cat_cols)
    if bin_columns_dict is not None:
        combined = bin_numerical_columns(combined,bin_columns_dict,inplace=True)

    cat_cols = get_cat_columns_by_type(combined)
    cat_cols = [cat_cols[i][0] for i in range(len(cat_cols))]
    num_cols = [col for col in combined.columns if col not in cat_cols]
    n_train = df.shape[0]
    n_test = df_test.shape[0]
   
    if remove_skewness:
        skewed_cols = combined[num_cols].apply(lambda x: skew(x)).sort_values(ascending=False)
        skewness = pd.DataFrame({'Skew' :skewed_cols})
        skewness_log = skewness[skewness > 4.0]
        skewness_other = skewness[skewness <= 4.0]
        skewed_features_log = skewness_log.index
        skewed_features_other = skewness_other.index
        lambda_ = 0.0
        for feature in skewed_features_log:
            combined[feature] = boxcox1p(combined[feature],lambda_)
        lambda_ = 0.15
        for feature in skewed_features_other:
            combined[feature] = boxcox1p(combined[feature],lambda_)
      
    combined,d = handle_missing_values(combined,cat_cols=cat_cols,num_cols=num_cols,inplace=True)
    
    combined = pd.get_dummies(combined, dummy_na=True)
    
    if scale_mapper is not None:
        map_f = [([c],scale_mapper) for c in num_cols]
        mapper = DataFrameMapper(map_f).fit(combined)
    else:
        mapper = None
        
    combined,_ = scale_num_cols(combined,mapper) 
    
    return combined,df,y,cat_cols,num_cols,test_id,n_train,n_test


In [4]:
PATH = "data/iowa_housing/"

In [5]:
df_raw = pd.read_csv(f'{PATH}train.csv', low_memory=False)
df_test = pd.read_csv(f'{PATH}test.csv', low_memory=False)
df = df_raw.copy()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [7]:
combined,df,y,cat_cols,num_cols,test_id,n_train,n_test = preprocess_df(df_train=df,df_test=df_test,id_col='Id',
                                                                      convert_to_cat_cols=['MoSold','YrSold','YearBuilt','GarageYrBlt','YearRemodAdd'],
                                                                      remove_skewness=False,
                                                                      )
print(combined.shape,len(cat_cols),len(num_cols),n_train,n_test)

C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype

(2919, 89) 48 31 1460 1459


In [8]:
cat_cols = get_cat_columns_by_type(df_raw)
cat_cols = [cat_cols[i][0] for i in range(len(cat_cols))]
num_cols = [col for col in df_raw.columns if col not in cat_cols]
skewed_cols = df_raw[num_cols].apply(lambda x: skew(x)).sort_values(ascending=False)
skewed_cols

MiscVal          24.451640
PoolArea         14.813135
LotArea          12.195142
3SsnPorch        10.293752
LowQualFinSF      9.002080
KitchenAbvGr      4.483784
BsmtFinSF2        4.250888
ScreenPorch       4.117977
BsmtHalfBath      4.099186
EnclosedPorch     3.086696
OpenPorchSF       2.361912
SalePrice         1.880941
BsmtFinSF1        1.683771
WoodDeckSF        1.539792
TotalBsmtSF       1.522688
MSSubClass        1.406210
1stFlrSF          1.375342
GrLivArea         1.365156
BsmtUnfSF         0.919323
2ndFlrSF          0.812194
OverallCond       0.692355
TotRmsAbvGrd      0.675646
HalfBath          0.675203
Fireplaces        0.648898
BsmtFullBath      0.595454
OverallQual       0.216721
MoSold            0.211835
BedroomAbvGr      0.211572
GarageArea        0.179796
YrSold            0.096170
FullBath          0.036524
Id                0.000000
GarageCars       -0.342197
YearRemodAdd     -0.503044
YearBuilt        -0.612831
LotFrontage            NaN
MasVnrArea             NaN
G

In [9]:
iqr_ranges = get_iqr_min_max(df,num_cols)
iqr_ranges

C:\Users\avinash.t\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3652: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


{'Id': (-728.5, 2189.5),
 'MSSubClass': (-55.0, 145.0),
 'LotFrontage': (nan, nan),
 'LotArea': (1481.5, 17673.5),
 'OverallQual': (2.0, 10.0),
 'OverallCond': (3.5, 7.5),
 'YearBuilt': (1885.0, 2069.0),
 'YearRemodAdd': (1911.5, 2059.5),
 'MasVnrArea': (nan, nan),
 'BsmtFinSF1': (-1068.375, 1780.625),
 'BsmtFinSF2': (0.0, 0.0),
 'BsmtUnfSF': (-654.5, 1685.5),
 'TotalBsmtSF': (42.0, 2052.0),
 '1stFlrSF': (118.125, 2155.125),
 '2ndFlrSF': (-1092.0, 1820.0),
 'LowQualFinSF': (0.0, 0.0),
 'GrLivArea': (158.625, 2747.625),
 'BsmtFullBath': (-1.5, 2.5),
 'BsmtHalfBath': (0.0, 0.0),
 'FullBath': (-0.5, 3.5),
 'HalfBath': (-1.5, 2.5),
 'BedroomAbvGr': (0.5, 4.5),
 'KitchenAbvGr': (1.0, 1.0),
 'TotRmsAbvGrd': (2.0, 10.0),
 'Fireplaces': (-1.5, 2.5),
 'GarageYrBlt': (nan, nan),
 'GarageCars': (-0.5, 3.5),
 'GarageArea': (-27.75, 938.25),
 'WoodDeckSF': (-252.0, 420.0),
 'OpenPorchSF': (-102.0, 170.0),
 'EnclosedPorch': (0.0, 0.0),
 '3SsnPorch': (0.0, 0.0),
 'ScreenPorch': (0.0, 0.0),
 'PoolArea

In [10]:
df['GrLivArea'].describe()

count    1460.000000
mean     1515.463699
std       525.480383
min       334.000000
25%      1129.500000
50%      1464.000000
75%      1776.750000
max      5642.000000
Name: GrLivArea, dtype: float64

In [11]:
df['GrLivArea'].median()

1464.0

In [12]:
df = combined[:n_train]
df_test = combined[n_train:]

X_train,X_valid,y_train,y_valid = train_test_split(df,y,test_size=0.15,
                                  stratify=df['OverallQual'],shuffle = True,random_state=20)
print('Data Ready')

Data Ready


In [13]:
rf_model = RandomForestRegressor(n_estimators=1500,n_jobs=-1,oob_score=True).fit(X_train.values,
                                                                                y_train)
print_mse(rf_model,X_train,X_valid,y_train,y_valid)

MSE Training set = 0.0027714576200914267, MSE Validation set = 0.018623891732581194, score Training Set = 0.9826270829573176, score on Validation Set = 0.8828690283948828
OOB Score = 0.8723089282016903


In [14]:
df_raw = pd.read_csv(f'{PATH}train.csv', low_memory=False)
df_test = pd.read_csv(f'{PATH}test.csv', low_memory=False)
df = df_raw.copy()
combined,df,y,cat_cols,num_cols,_,n_train,n_test = preprocess_df(df_train=df,df_test=df_test,
                                                                       drop_target=True,
                                                                       id_col='Id',
                                                                       log_y=True,
                                                                       convert_to_cat_cols=['MoSold','YrSold','YearBuilt','GarageYrBlt','YearRemodAdd'],
                                                                       remove_skewness=True
                                                                       )
print(combined.shape,len(cat_cols),len(num_cols),n_train,n_test)

C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype

(2919, 89) 48 31 1460 1459


In [15]:
df = combined[:n_train]
df_test = combined[n_train:]

X_train,X_valid,y_train,y_valid = train_test_split(df,y,test_size=0.15,
                                  stratify=df['OverallQual'],shuffle = True,random_state=20)
print('Data Ready')

Data Ready


In [16]:
rf_model = RandomForestRegressor(n_estimators=1500,n_jobs=-1,oob_score=True).fit(X_train.values,
                                                                                y_train)
print_mse(rf_model,X_train,X_valid,y_train,y_valid)

MSE Training set = 0.00278768496979403, MSE Validation set = 0.018461815297837327, score Training Set = 0.982525361611062, score on Validation Set = 0.8838883733603954
OOB Score = 0.8728831316993569


In [17]:
df_raw = pd.read_csv(f'{PATH}train.csv', low_memory=False)
df_test = pd.read_csv(f'{PATH}test.csv', low_memory=False)
df = df_raw.copy()
combined,df,y,cat_cols,num_cols,_,n_train,n_test = preprocess_df(df_train=df,df_test=df_test,
                                                   drop_target=True,
                                                   id_col='Id',
                                                   log_y=True,
                                                   convert_to_cat_cols=['MoSold','YrSold','YearBuilt','GarageYrBlt','YearRemodAdd'],
                                                   remove_skewness=True,
                                                   scale_mapper=RobustScaler()
                                                  )

In [18]:
df = combined[:n_train]
df_test = combined[n_train:]

X_train,X_valid,y_train,y_valid = train_test_split(df,y,test_size=0.15,
                                  stratify=df['OverallQual'],shuffle = True,random_state=20)
print('Data Ready')

Data Ready


In [19]:
rf_model = RandomForestRegressor(n_estimators=1500,n_jobs=-1,oob_score=True).fit(X_train.values,
                                                                                y_train)
print_mse(rf_model,X_train,X_valid,y_train,y_valid)

MSE Training set = 0.002758727896118088, MSE Validation set = 0.0185844347298253, score Training Set = 0.9827068793925802, score on Validation Set = 0.8831171847488715
OOB Score = 0.8725569742158581


In [20]:
important_features = pd.Series(data=rf_model.feature_importances_,index=X_train.columns)
important_features.sort_values(ascending=False,inplace=True)
important_features.head(50)

OverallQual      0.543229
GrLivArea        0.115579
TotalBsmtSF      0.049409
GarageArea       0.036709
GarageCars       0.035700
BsmtFinSF1       0.019961
1stFlrSF         0.019232
YearBuilt        0.014846
CentralAir       0.012402
LotArea          0.012329
OverallCond      0.011713
MSZoning         0.009453
YearRemodAdd     0.007921
2ndFlrSF         0.007491
GarageFinish     0.007137
BsmtUnfSF        0.005519
Neighborhood     0.005182
LotFrontage      0.004602
GarageYrBlt      0.004329
Fireplaces       0.004012
OpenPorchSF      0.003844
MoSold           0.003343
WoodDeckSF       0.003294
GarageType       0.003279
TotRmsAbvGrd     0.003262
MasVnrArea       0.003262
FullBath         0.003122
EnclosedPorch    0.002709
KitchenQual      0.002613
ExterCond        0.002579
BsmtQual         0.002559
BsmtFinType1     0.002334
MSSubClass       0.002142
SaleCondition    0.002096
BedroomAbvGr     0.002016
LotShape         0.001968
Exterior1st      0.001837
LandContour      0.001802
BsmtExposure

In [21]:
combined['CentralAir'].value_counts()

1    2723
0     196
Name: CentralAir, dtype: int64

In [22]:
combined['OverallQual'].value_counts()

1.109946    825
1.172893    731
1.225196    600
1.269755    342
1.031562    226
1.308446    107
0.929025     40
1.342554     31
0.784059     13
0.547945      4
Name: OverallQual, dtype: int64

In [23]:
combined['OverallCond'].value_counts()

1.109946    1645
1.172893     531
1.225196     390
1.269755     144
1.031562     101
0.929025      50
1.308446      41
0.784059      10
0.547945       7
Name: OverallCond, dtype: int64

In [24]:
df_raw = pd.read_csv(f'{PATH}train.csv', low_memory=False)
df_test = pd.read_csv(f'{PATH}test.csv', low_memory=False)
df = df_raw.copy()
combined,df,y,cat_cols,num_cols,_,n_train,n_test = preprocess_df(df_train=df,df_test=df_test,
                                                                       drop_target=True,
                                                                       id_col='Id',
                                                                       log_y=True,
                                                                       convert_to_cat_cols=['CentralAir','MoSold','YrSold','YearBuilt','GarageYrBlt','YearRemodAdd'],
                                                                       remove_skewness=True,
                                                                       scale_mapper=RobustScaler(),
                                                                       bin_columns_dict={'OverallQual':(1,11,10),'OverallCond':(1,11,10)}
                                                                   )



In [25]:
df = combined[:n_train]
df_test = combined[n_train:]

X_train,X_valid,y_train,y_valid = train_test_split(df,y,test_size=0.15,
                                  stratify=df['OverallQual'],shuffle = True,random_state=20)
rf_model = RandomForestRegressor(n_estimators=1500,n_jobs=-1,oob_score=True).fit(X_train.values,
                                                                                y_train)
print_mse(rf_model,X_train,X_valid,y_train,y_valid)

MSE Training set = 0.00271991649243513, MSE Validation set = 0.020536781195579175, score Training Set = 0.9827587406341884, score on Validation Set = 0.8784883761916227
OOB Score = 0.8729013120926431


In [26]:
df_raw = pd.read_csv(f'{PATH}train.csv', low_memory=False)
df_test = pd.read_csv(f'{PATH}test.csv', low_memory=False)
df = df_raw.copy()
combined,df,y,cat_cols,num_cols,test_id,n_train,n_test = preprocess_df(df_train=df,df_test=df_test,
                                                                       drop_target=True,
                                                                       id_col='Id',
                                                                       log_y=True,
                                                                       convert_to_cat_cols=['MoSold','YrSold','YearBuilt','GarageYrBlt','YearRemodAdd'],
                                                                       remove_skewness=True,
                                                                       scale_mapper=RobustScaler(),
                                                                       bin_columns_dict={'OverallQual':(1,11,10),'OverallCond':(1,11,10)}
                                                                   )


map_f = [([c],MinMaxScaler()) for c in combined.columns if is_numeric_dtype(combined[c])]
mapper = DataFrameMapper(map_f).fit(combined)
combined,_ = scale_num_cols(combined,mapper)

C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 w

In [27]:
df = combined[:n_train]
df_test = combined[n_train:]

X_train,X_valid,y_train,y_valid = train_test_split(df,y,test_size=0.15,
                                  stratify=df['OverallQual'],shuffle = True,random_state=20)
rf_model = RandomForestRegressor(n_estimators=1500,n_jobs=-1,oob_score=True).fit(X_train.values,
                                                                                y_train)
print_mse(rf_model,X_train,X_valid,y_train,y_valid)

MSE Training set = 0.0027029021305786994, MSE Validation set = 0.020450956055751777, score Training Set = 0.9828665928519041, score on Validation Set = 0.8789961846940695
OOB Score = 0.8722875868118509


In [28]:
df_raw = pd.read_csv(f'{PATH}train.csv', low_memory=False)
df_test = pd.read_csv(f'{PATH}test.csv', low_memory=False)
df = df_raw.copy()
combined,df,y,cat_cols,num_cols,test_id,n_train,n_test = preprocess_df(df_train=df,df_test=df_test,
                                                                       drop_target=True,
                                                                       id_col='Id',
                                                                       log_y=True,
                                                                       convert_to_cat_cols=['MoSold','YrSold','YearBuilt','GarageYrBlt','YearRemodAdd'],
                                                                       remove_skewness=True,
                                                                       scale_mapper=RobustScaler(),
                                                                       bin_columns_dict={'OverallQual':(1,11,10),'OverallCond':(1,11,10)}
                                                                   )


combined,_ = scale_num_cols(combined,None)

C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avinash.t\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype

In [29]:
df = combined[:n_train]
df_test = combined[n_train:]

X_train,X_valid,y_train,y_valid = train_test_split(df,y,test_size=0.15,
                                  stratify=df['OverallQual'],shuffle = True,random_state=20)
rf_model = RandomForestRegressor(n_estimators=1500,n_jobs=-1,oob_score=True).fit(X_train.values,
                                                                                y_train)
print_mse(rf_model,X_train,X_valid,y_train,y_valid)

MSE Training set = 0.002739042997900299, MSE Validation set = 0.020204680653249724, score Training Set = 0.9826374997643295, score on Validation Set = 0.8804533421608163
OOB Score = 0.872311738796436
